# Hypotheses:
- There is a time-encoded relationship between similarity of orgabnoid development and fetal skin development

# Assumptions:
- Measurable differences between both in-vivo and in-vitro systems are attributed to two parameters:

     **Measureable Parameters**
    - 1. There is a relationship between development time and transcriptomic representation of cell-states
    - 2. There is a relationship between time and the cell-state distribution of both in-vitro and in-vivo systems

# Objectives:

**2 main goals are to:**
- Find some time encoding for transcriptomic difference by cell-state
- Find if two timepoints have similar distributions
 
# Approach:
- Integrate all data using an ldVAE/VAE, if there is indeed some time-specific RNA variation, the model should hopefully learn this, unless we did a very bad job in accoiunting for batch etc.
- Train an SVM/LR on the time-encoded cell-state information provided by the low-dimensional embeddings
- Find the cell-state-time specific probabillisitc projection probabilities
    
    **Time_vivo vs time_vitro comparisons**
    - We want to find some combination of global cell-state projection probabilities
    - We will weigh these probabilities by some measure of distributional similarity between the model and query data (likelihood function)
    - Aggregate the posterior projected probabilities for the labels which are matched in training-time for each time-point in the query data.
    - Options: 
        - project trained model on each target timepoint individually
        - project trained model across entire fetal/organoid data, then disentangle the probabilities. (aggregated probabilities per cell will still sum to one, we can take the median aggregated probabilities for each cell-state-time label for a given class.)
    - Optionally normalise such that summing the aggregated probabilities for a specific training-timepoint should = 1. 
    
    - Since we pass a matrix of 'prior' probabilities (output for LR/SVM) the corrected probability is a single value per celltype/time encoding

    **Individual cell-states by time comparisons**
    - Here we no longer want to penalise the pribabilities by distribution as we are interested in only the similarity of the transcriptomic landscape
    - We take the organised probability by cell-state with time encodings and find the median projection probability for matching or predicted cellstates in the query data

**Liklihood function**
- Bayesian model contructed as a loop for each training time-point (to keeop the observatrions standard). 
- Prior = SVM/LR probabilities per cellstate-time (weighted by reciprocal of donor and cell-state availability). Provided as seperate matrices for labels by time for each loop
- Observation = original observed probability in the training data (skin/organoid)
- Output: one posterior probability per cell-state-time label



Write code to apply logistic regression to train a model on expression data from single-cell RNA-seq data in the anndata format. The model should learn time and celltype information and project this onto another data. Use a bayesian framework to output posterior probabilities based on the output from the logistic regression model as priors and the actual celltype by time distribution as observations. 

# Fskin approach
- Train SVM on time-encoded celltype representation of the Fskin data
- Create a leverage score sampled set of cells for each celltrype in the SVM model
- Do not bother subsetting each data to only cells present in the organoid data

- Not subsetting to only cell-tyoes present in all data could introduce issues for cross-compartability, but presents the most impartial view of probabillistic correspondecne

# Emprical Bayes framework for SVM deconvolution and classification of single cell and bulk data

### Motivations:
- Single cell label transfer tasks are a complex task where experimental design parameters or other confounders such as cell-state distribution/sampling are often not accounted for. 
- Linea

- Include experimental design and known biology into the analytical framework:
    - Ability to incoperate prior information and uncertainties on cell type distribution into the modelling process, helps us identify the liklihood of cell-state matching assuming the query data is of the same procerssing
    - If the query data is not of the same system/distribution, the liklihood function penalises the projection  
    - By incorporating priors, the Bayesian probabilistic SVM can explicitly account for uncertainties in the cell-type distribution, which is especially valuable when dealing with noisy or incomplete data. This allows the model to provide probabilistic predictions and quantify the uncertainty associated with each prediction.
    
- Why SVM?
    - Compared to linear regression methods like logistic regression, Bayesian probabilistic SVM offers a more flexible and robust approach that can better capture complex relationships and uncertainties in the data. 
    - **Less sensitive to feature scaling:** SVM is generally less sensitive to feature scaling than logistic regression making it suitable for cross-modality learning and projection. While logistic regression's performance can be affected by different feature scales, SVM typically normalizes the feature values during the training process, making it less dependent on feature scaling.
    - **Non-linear relationships:** SVM can handle non-linear relationships between the input variables and the target variable by using different kernel functions. This flexibility allows SVM to capture more complex patterns and make non-linear decision boundaries, while logistic regression assumes a linear relationship between the features and the target variable.
    - **Robustness to outliers:** SVM is generally more robust to outliers in the data compared to logistic regression. SVM aims to maximize the margin between classes, which helps it to be less influenced by outliers that might significantly affect the decision boundary of logistic regression.
    - **Effective in high-dimensional spaces:** SVM is particularly effective in high-dimensional spaces, where the number of features is large compared to the number of samples. SVM uses a subset of the training points called support vectors to determine the decision boundary, making it more memory-efficient and computationally efficient in high-dimensional scenarios compared to logistic regression.



### Architectural summary:

- **Support Vector Machine framework**
    - We train a single binary classifier for each class versus all. 
    - Features are transformed by the Nystroem approximation transformer for the RNF kernel based on a set of randomly sampled features (sampling and eigen decomposition)
    - Nystroem transofmred features are input as training data into the SVM, the RNF kernel computes node-node relationships in the n-1 dimensions
    - Bayesian oprimisation is used to find an optimal Gamma and Sparcity hyper parameter
    - Platt scaled-distance to the hyperplane is used to estimate probabilites belonging to each class (fitting a lgistic regression model which aims to learn the mapping between the decision scores and the true class probabilities per pariwise couplings. )


- **Emperical Bayesian framework:**
    - Model employs an iterative hypergeomtric sampling step per batch which provides refined cell-type distribution priors which are robust to oversampled/undersampled populations - undersampled populations will have a lower prob, thus if the model returns a low probability observation, we do not penalise it.
    - Model employs a set of prior sampling-weights which are the reicprocal of donors per cell tpye. This penalises celltypes which are observed in fewer donors. 
    - Observed probabilies that are provded by the SVM model 
    - Posterior probabilities represent the observed probability multiplied by the liklihood of observing the distribution given a weighted prior
    

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import pymc3 as pm
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from skopt import BayesSearchCV
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import hypergeom
from scipy.stats import chi2_contingency
from sklearn.kernel_approximation import Nystroem
from numpy import inf
from itertools import chain
from sklearn.metrics import classification_report

In [ ]:
# Bayesian ridge regression
# Probabillistic SVM

# prototype code:

Instead, it uses a probabilistic SVM classifier to estimate the probability that each single-cell belongs to a particular cell type. The SVM classifier is trained on the single-cell data, where each cell type is treated as a separate binary classification problem using a one-vs-rest approach. The resulting probability matrix is then used as input to a Bayesian deconvolution model, which assumes that each bulk sample is a linear combination of the gene expression profiles of the individual cell types, weighted by their relative contributions. The Bayesian model estimates the cell type proportions and gene expression profiles that best explain the observed bulk RNA sequencing data, using prior distributions and a likelihood function. The posterior distribution of the model parameters is then sampled using MCMC methods to obtain a set of posterior samples.

#### method summary

Load single-cell sequencing data: The first step is to load the single-cell sequencing data using Scanpy. Let $X$ be the gene expression matrix for the single-cell data, where each row corresponds to a gene and each column corresponds to a single cell.

Extract gene expression profiles: Next, extract the gene expression profiles for each cell by converting the Scanpy object to a dense matrix. Let $x_i$ denote the gene expression profile for cell $i$, where $x_i$ is a column vector of length $n$, the number of genes.

Load and normalize bulk RNA sequencing data: Load the bulk RNA sequencing data and normalize it to obtain a matrix $Y$ of shape $m \times n$, where $m$ is the number of samples (e.g., tissues) and $n$ is the number of genes. Let $y_j$ denote the gene expression profile for sample $j$, where $y_j$ is a row vector of length $n$. Normalize $Y$ by dividing each row by its sum, so that the rows sum to 1 and represent proportions.

Train probabilistic SVM classifier: Train a probabilistic support vector machine (SVM) classifier on the single-cell data, using the cell types as the target variable. Let $C$ be the set of cell types, and let $y_i$ denote the cell type for cell $i$. We use a one-vs-rest approach, where each cell type is treated as a separate binary classification problem. Let $p_i(c)$ denote the probability that cell $i$ belongs to cell type $c$, as estimated by the SVM classifier. We obtain a probability matrix $P$ of shape $k \times m$, where $k$ is the number of cell types and $m$ is the number of bulk samples, by applying the SVM classifier to the bulk data.

### RBF kernel trick
Non-linearity: The RBF kernel allows SVM to handle non-linear relationships between input features and class labels. It can capture complex patterns and decision boundaries that linear models cannot. This flexibility makes it well-suited for datasets where the relationship between features and classes is non-linear.

Implicit feature mapping: The RBF kernel implicitly maps the input features into a higher-dimensional feature space, where non-linear relationships may become linear. This mapping allows SVM to effectively separate classes that are not linearly separable in the original feature space.

Flexibility in hyperparameter tuning: The RBF kernel has an additional hyperparameter called gamma, which determines the kernel's influence range. A smaller gamma value leads to a wider influence range and smoother decision boundaries, while a larger gamma value results in narrower influence range and more complex decision boundaries. This flexibility allows fine-tuning of the SVM model to capture the desired level of complexity in the data.

### Bayesian liklihood
Specify prior distributions and likelihood function: Specify a Bayesian model for deconvolving the bulk RNA sequencing data. We assume that each bulk sample is a linear combination of the gene expression profiles of the individual cell types, weighted by their relative contributions. Let $a_c$ be the prior probability that cell type $c$ is present in the bulk sample. We assume a Dirichlet prior for $a$, with concentration parameter $\alpha$ set to 1 for a non-informative prior. Let $b_{c,j}$ be the gene expression level for gene $j$ in cell type $c$. We assume a normal prior for $b_{c,j}$, with mean 0 and variance 1 for a non-informative prior. We then model the bulk RNA sequencing data as a multivariate normal distribution, with mean vector $\mu = \sum_{c=1}^k a_c b_c$ and covariance matrix $\Sigma = I$, where $b_c$ is the column vector of gene expression levels for cell type $c$.

Sample from posterior distribution: Use Markov chain Monte Carlo (MCMC) methods to sample from the posterior distribution of the model parameters, given the data. We use the PyMC3 library to perform the MCMC sampling. Let $\theta = (a,b)$ denote the set of model parameters. We obtain a set of $N$ posterior samples ${\theta_i}_{i=1}^N$.

Assess convergence and quality of samples: Check the convergence and quality of the posterior samples using diagnostic plots, such as trace plots and autocorrelation plots. This step ensures that the

# Define training data
- Let's use the non-linear VAE integration in the first instance
- data is capped at 15,000 features based on dispersion

In [ ]:
datasets = {
    "ldvae_integrated": "/nfs/team205/ig7/projects/fetal_skin/160523_probabillistic_projection_organoid_adt_fetl/data/A1_V1_linear_ldvae_scvi_ski_updated_org_adt_build_donor_source_corrected_160523_raw_concatenated_ldvae_xscvi_features.h5ad",
    "VAE_integrated": "/nfs/team205/ig7/projects/fetal_skin/160523_probabillistic_projection_organoid_adt_fetl/data/A1_V1_ldvae_scvi_ski_updated_org_adt_build_donor_source_corrected_160523_raw_concatenated_xscvi_features.h5ad",
    "full_feature_space": "/nfs/team205/ig7/projects/fetal_skin/160523_probabillistic_projection_organoid_adt_fetl/data/A1_V1_ldvae_scvi_ski_updated_org_adt_build_donor_source_corrected_160523_raw_concatenated_full_features.h5ad",
}
adata_int = sc.read(datasets["VAE_integrated"], backed="r")

# Train on the Fskin compartment of the integrated data

In [ ]:
data_select_col = "dataset_merge"
data_select = "-organoid"
train_indexer = adata_int.obs[adata_int.obs.index.str.contains(data_select)].index
adata = adata_int[adata_int.obs[data_select_col].isin([data_select])].to_memory()
# adata = adata_int[adata_int.obs.index.isin(train_indexer)].to_memory()
sc_data = adata

# Check for timepoints where <3 of specific celltypes are there

In [ ]:
adata.obs["age_annot"] = (
    adata.obs["pcw"].astype(str) + "_" + adata.obs["annot"].astype(str)
)
ct = adata.obs.groupby(["age_annot"]).apply(len)
print(ct[ct < 3])
ct = list(ct[ct < 3].index)
ct

In [ ]:
train_var = "age_annot"
train_x_partition = "X_scvi"
bless_partition = "X_scvi"
use_nystroem = False
apply_heuristic_vars = [
    "annot",
    "pcw",
]  # this list of 2 variables defines a category to split the data by and a category to ensure good distributional representation

In [ ]:
adata.obs.groupby([train_var]).apply(len)

In [ ]:
sc_data

In [ ]:
adata.obs

In [ ]:
sc_data.obs["pcw"].unique()

In [ ]:
# # sc.pp.filter_cells(sc_data, min_genes=200)
# # sc.pp.filter_genes(sc_data, min_cells=3)
# sc.pp.normalize_per_cell(sc_data, counts_per_cell_after=1e4)
# sc.pp.log1p(sc_data)
# sc.pp.highly_variable_genes(sc_data, min_mean=0.1, max_mean=4)
# # adata_mac = nameadata_mac adata_mac.var['highly_variable']]
# sc.pp.scale(sc_data, max_value=10)
# #%% PCA
# sc.tl.pca(sc_data, n_comps=50,use_highly_variable=True)

In [ ]:
sc_data.obs["annot"]

# Apply leverage score sampling
### Special heuristic rules::
- We sample to a minimum number of vertices (0.1,0.05)
- We apply sampling per celltype in this instance to try and learn a time encoding
- Distribution of celltypes by time must pass chi-square test

In [ ]:
# apply Bless
import bless as bless
from sklearn.gaussian_process.kernels import RBF
from numpy import arange
from sklearn.model_selection import RepeatedKFold
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

random_state = 24
qbar = 2
max_qbar = 10
lam_final = 2
min_prop = 0.01
min_cells_per_var = 3

if apply_heuristic_vars == False:
    print("qbar param: {} , lambda param: {}".format(qbar, lam_final))
    # If latent rep is provided, randomly sample data in spatially aware manner for initialisation
    r = np.random.RandomState(random_state)
    if bless_partition in sc_data.obsm.keys():
        tune_train_x = sc_data.obsm[bless_partition][:]
        lvg = bless.bless(
            tune_train_x,
            RBF(length_scale=20),
            lam_final=lam_final,
            qbar=qbar,
            random_state=r,
            H=10,
            force_cpu=True,
        )
        adata_tuning = sc_data[lvg.idx]
        tune_train_x = adata_tuning.obsm[bless_partition][:]
        print("Sketched data is {} long".format(len(adata_tuning.obs)))

    else:
        tune_train_x = sc_data.X
        lvg = bless.bless(
            tune_train_x,
            RBF(length_scale=20),
            lam_final=lam_final,
            qbar=qbar,
            random_state=r,
            H=10,
            force_cpu=True,
        )
        adata_tuning = sc_data[lvg.idx]
        tune_train_x = adata_tuning.obsm[sketch_obsm][:]
        print("Sketched data is {} long".format(len(adata_tuning.obs)))

    while (
        (
            len(
                list(
                    set(list(adata_tuning.obs[train_var].unique()))
                    ^ set(list(sc_data.obs[train_var].unique()))
                )
            )
            > 0
        )
        or (min(adata_tuning.obs.groupby([train_var]).apply(len)) < 5)
        or (qbar == max_qbar)
        or (len(lvg.idx) / len(sc_data.obs) < min_prop)
    ):
        #     lam_final = lam_final + 1
        qbar = qbar + 1
        print("qbar param: {} , lambda param: {}".format(qbar, lam_final))
        # If latent rep is provided, randomly sample data in spatially aware manner for initialisation
        r = np.random.RandomState(random_state)
        if bless_partition in sc_data.obsm.keys():
            tune_train_x = sc_data.obsm[bless_partition][:]
            lvg = bless.bless(
                tune_train_x,
                RBF(length_scale=20),
                lam_final=lam_final,
                qbar=qbar,
                random_state=r,
                H=10,
                force_cpu=True,
            )
            adata_tuning = sc_data[lvg.idx]
            tune_train_x = adata_tuning.obsm[bless_partition][:]
            print("Sketched data is {} long".format(len(adata_tuning.obs)))
        else:
            tune_train_x = sc_data.X
            lvg = bless.bless(
                tune_train_x,
                RBF(length_scale=20),
                lam_final=lam_final,
                qbar=qbar,
                random_state=r,
                H=10,
                force_cpu=True,
            )
            adata_tuning = sc_data[lvg.idx]
            tune_train_x = adata_tuning.obsm[sketch_obsm][:]
            print("Sketched data is {} long".format(len(adata_tuning.obs)))


else:
    heuristic_param_1 = apply_heuristic_vars[0]
    heuristic_param_2 = apply_heuristic_vars[1]
    tuned_indx = {}

    for class_type in list(sc_data.obs[heuristic_param_1].unique()):
        print(class_type)
        sc_data_tmp = sc_data[sc_data.obs[heuristic_param_1].isin([class_type])]
        ct = sc_data_tmp.obs.groupby(["age_annot"]).apply(len)
        ct = list(ct[ct < min_cells_per_var].index)
        if len(ct) > 0:
            print(
                "There are cells which are not significantly present at all stages of heuristic var 2 {}".format(
                    str(ct)
                )
            )
            print("removing these cells from training")
            sc_data_tmp = sc_data_tmp[~sc_data_tmp.obs["age_annot"].isin(ct)]

        print("qbar param: {} , lambda param: {}".format(qbar, lam_final))
        # If latent rep is provided, randomly sample data in spatially aware manner for initialisation
        r = np.random.RandomState(random_state)
        if bless_partition in sc_data_tmp.obsm.keys():
            print("Leverage score from: {}".format(bless_partition))
            tune_train_x = sc_data_tmp.obsm[bless_partition][:]
            lvg = bless.bless(
                tune_train_x,
                RBF(length_scale=10),
                lam_final=lam_final,
                qbar=qbar,
                random_state=r,
                H=5,
                force_cpu=True,
            )

            adata_tuning = sc_data_tmp[lvg.idx]
            tuned_indx[class_type] = adata_tuning.obs.index
            # tune_train_x = adata_tuning.obsm[bless_partition][:]
            print("Sketched data is {} long".format(len(adata_tuning.obs)))

        else:
            tune_train_x = sc_data_tmp.X
            lvg = bless.bless(
                tune_train_x,
                RBF(length_scale=10),
                lam_final=lam_final,
                qbar=qbar,
                random_state=r,
                H=10,
                force_cpu=True,
            )
            adata_tuning = sc_data_tmp[lvg.idx]
            tuned_indx[class_type] = adata_tuning.obs.index
            # tune_train_x = adata_tuning.obsm[sketch_obsm][:]
            print("Sketched data is {} long".format(len(adata_tuning.obs)))

        # rules
        # Sample must contain all heuristic param 2
        # Sample must have >3 of the min group
        # Sample must have > min_prop
        # or we reach max sampling
        while (
            (
                len(
                    list(
                        set(list(adata_tuning.obs[heuristic_param_2].unique()))
                        ^ set(list(sc_data_tmp.obs[heuristic_param_2].unique()))
                    )
                )
                > 0
            )
            or (
                min(adata_tuning.obs.groupby([heuristic_param_2]).apply(len))
                < min_cells_per_var
            )
            or (qbar == max_qbar)
            or (len(lvg.idx) / len(sc_data_tmp.obs) < min_prop)
        ):
            #     lam_final = lam_final + 1
            if (
                len(
                    list(
                        set(list(adata_tuning.obs[heuristic_param_2].unique()))
                        ^ set(list(sc_data_tmp.obs[heuristic_param_2].unique()))
                    )
                )
                > 0
            ):
                print("missing heuristic 2 variables!")
            if min(adata_tuning.obs.groupby([heuristic_param_2]).apply(len)) < 3:
                print("One heuristic 2 variable is numbered < 3!")
            if len(lvg.idx) / len(sc_data_tmp.obs) < min_prop:
                print(
                    "Sampled population is < than selected min_prop var {}".format(
                        min_prop
                    )
                )
            qbar = qbar + 1
            lam_final = lam_final + 1
            print("qbar param: {} , lambda param: {}".format(qbar, lam_final))
            # If latent rep is provided, randomly sample data in spatially aware manner for initialisation
            r = np.random.RandomState(random_state)
            if bless_partition in sc_data_tmp.obsm.keys():
                tune_train_x = sc_data_tmp.obsm[bless_partition][:]
                lvg = bless.bless(
                    tune_train_x,
                    RBF(length_scale=10),
                    lam_final=lam_final,
                    qbar=qbar,
                    random_state=r,
                    H=5,
                    force_cpu=True,
                )
                adata_tuning = sc_data_tmp[lvg.idx]
                tuned_indx[class_type] = adata_tuning.obs.index
                # tune_train_x = adata_tuning.obsm[bless_partition][:]
                print("Sketched data is {} long".format(len(adata_tuning.obs)))
            else:
                tune_train_x = sc_data_tmp.X
                lvg = bless.bless(
                    tune_train_x,
                    RBF(length_scale=10),
                    lam_final=lam_final,
                    qbar=qbar,
                    random_state=r,
                    H=5,
                    force_cpu=True,
                )
                adata_tuning = sc_data_tmp[lvg.idx]
                tuned_indx[class_type] = adata_tuning.obs.index
                # tune_train_x = adata_tuning.obsm[sketch_obsm][:]
                print("Sketched data is {} long".format(len(adata_tuning.obs)))

In [ ]:
missing_cat = list(
    set(list(sc_data.obs["annot"].unique())) ^ set(list(tuned_indx.keys()))
)
if len(missing_cat) > 0:
    print("Missing categories! {}".format(str(missing_cat)))
sampled_indx = list(set(chain(*tuned_indx.values())))
adata_tuning = sc_data[sc_data.obs.index.isin(sampled_indx)]
adata_tuning

In [ ]:
# Step 1: Load single-cell sequencing data using Scanpy
# sc_data = sc.read('/nfs/team205/ig7/projects/SCC_nano_string/khavari.h5ad')
# sc_data = sc_data[sc_data.obs['patient'].isin(['P1'])]
# Step 2: Extract gene expression profiles for each cell
sc_data = adata_tuning.copy()
sc_expr = sc_data.X.todense()
sc_expr = np.array(sc_expr)
sc_labels = sc_data.obs[train_var]

In [ ]:
sc_expr

In [ ]:
sc_labels.unique()

# Train probabilistic SVM classifier with nystroeam approximation
- Bayesian optimisation


#### Nystroem approximation transformer
- By using the Nystroem approximation transformer, the SVM model can effectively capture the non-linear relationships in the gene expression data by projecting it into a higher-dimensional feature space. This enables the SVM to learn complex decision boundaries and improve the classification performance compared to using a linear kernel.

- Nystroem approximation transformer is a feature mapping technique that approximates the RBF (Radial Basis Function) kernel matrix for the gene expression data. It transforms the original high-dimensional gene expression data into a lower-dimensional feature space, while still capturing the non-linear relationships present in the data.

- The Nystroem approximation is based on the concept of random Fourier features, where it approximates the kernel matrix by computing a subset of randomly selected samples (landmark points) from the input data. These landmark points are then used to construct an approximate feature representation of the data.

- Landmark Selection:
    Randomly select a subset of samples (landmarks) from the original data. The number of landmarks is determined by the n_components parameter.
    The landmarks can be chosen uniformly at random or using other sampling strategies like k-means clustering.
    
- Feature Transformation:
    Compute the RBF kernel matrix between the original data and the selected landmarks.
    Perform eigenvalue decomposition on the kernel matrix to obtain the eigenvectors and eigenvalues.
    Use the eigenvectors corresponding to the largest eigenvalues as the transformed features.
    Scale the transformed features by the square root of the corresponding eigenvalues.

#### Feature importance
One common method for measuring feature importance in SVM models is to use the absolute value of the weights assigned to each feature in the SVM model. The higher the weight, the more important the feature is in predicting the outcome.

In scikit-learn, the coef_ attribute of the SVC class can be used to obtain the coefficients of the SVM model. The coefficients are stored in an array of shape (n_classes, n_features) for multi-class classification problems or (1, n_features) for binary classification problems.

Here's an example code snippet that demonstrates how to obtain and sort the feature weights from an SVM model:

#### get feature weights from SVM model
weights = np.abs(clf.coef_[0])

#### sort features by weight
sorted_indices = np.argsort(weights)[::-1]

#### print feature weights in descending order
for i in sorted_indices:
    print(f'Feature {i}: {weights[i]}')
    

In [ ]:
use_nystroem = False

In [ ]:
# Train probabilistic SVM classifier
# The fit time scales at least quadruple with the number of samples and may be impractical beyond tens of thousands of samples.

# Hyperparameter optimization using Bayesian optimization
param_space = {"C": (0.1, 10.0, "log-uniform"), "gamma": (0.01, 10.0, "log-uniform")}

if use_nystroem == True:
    print(
        "Creating a approximate rbf kernel representation using the Nystroem transformer"
    )
    # Create Nystroem approximation transformer
    nystroem = Nystroem(
        kernel="rbf", n_components=1000
    )  # Adjust the number of components as needed
    # Transform the data using Nystroem approximation
    nystroem_features = nystroem.fit_transform(sc_expr)

    print(
        "Bayesian optimisation for regularisation and gamma parameters of the rbf kernel"
    )
    svm = SVC(
        kernel="rbf", probability=True
    )  # RBF kernel uses kernal trick to project data into an infinite dimensional space and finds the best hyperplane per class
    opt_model = BayesSearchCV(
        svm, param_space, n_iter=30, cv=3, scoring="f1_weighted", n_jobs=-1
    )
    opt_model.fit(nystroem_features, sc_labels)
    # Best hyperparameters
    best_params = opt_model.best_params_
    print("Fitting SVM model")
    # Train SVM classifier with RBF kernel using the best hyperparameters
    svm = SVC(kernel="rbf", probability=True, **best_params)
    svm.fit(nystroem_features, sc_labels)

else:
    print(
        "Warning! Proceeding without Nystroem transformer, increased samples here can quadruple the compute time!"
    )
    print(
        "Bayesian optimisation for regularisation and gamma parameters of the rbf kernel"
    )
    svm = SVC(
        kernel="rbf", probability=True
    )  # RBF kernel uses kernal trick to project data into an infinite dimensional space and finds the best hyperplane per class
    opt_model = BayesSearchCV(
        svm, param_space, n_iter=10, cv=3, scoring="f1_weighted", n_jobs=-1
    )
    opt_model.fit(sc_expr, sc_labels)
    # Best hyperparameters
    best_params = opt_model.best_params_
    print("Fitting SVM model")
    # Train SVM classifier with RBF kernel using the best hyperparameters
    svm = SVC(kernel="rbf", probability=True, **best_params)
    svm.fit(sc_expr, sc_labels)

# Plot Bayesian optimization iterations
plt.figure(figsize=(10, 6))
plt.plot(
    range(len(opt_model.cv_results_["mean_test_score"])),
    opt_model.cv_results_["mean_test_score"],
)
plt.xlabel("Iteration")
plt.ylabel("Mean F1_weighted Test Score")
plt.title("Bayesian Optimization Iterations")
# Find the best iteration
best_iteration = np.argmax(opt_model.cv_results_["mean_test_score"])
best_score = opt_model.cv_results_["mean_test_score"][best_iteration]
# Mark the best iteration
plt.scatter(
    best_iteration,
    best_score,
    marker="o",
    color="red",
    label="Best Iteration C:{} gamma:{}".format(best_params["C"], best_params["gamma"]),
)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
report = classification_report(
    sc_labels, svm.predict(nystroem.transform(sc_expr)), output_dict=True
)
report = pd.DataFrame(report).T
report

In [ ]:
pred_out = pd.DataFrame(
    svm.predict_proba(nystroem.transform(sc_expr)),
    index=sc_data.obs.index,
    columns=svm.classes_,
)
pred_out["predicted"] = svm.predict(nystroem.transform(sc_expr))
pred_out["orig_labels"] = sc_labels

In [ ]:
pred_out

In [ ]:
# Import seaborn
import seaborn as sns

In [ ]:
model_mean_probs = (
    pred_out.loc[:, pred_out.columns != "predicted"].groupby("orig_labels").median()
)
model_mean_probs = model_mean_probs * 100
# model_mean_probs = model_mean_probs.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
crs_tbl = model_mean_probs.copy()
# Sort df columns by rows
index_order = list(crs_tbl.max(axis=1).sort_values(ascending=False).index)
col_order = list(crs_tbl.max(axis=0).sort_values(ascending=False).index)
crs_tbl = crs_tbl.loc[index_order]
crs_tbl = crs_tbl[col_order]
# Plot_df_heatmap(crs_tbl, cmap='coolwarm', rotation=90, vmin=20, vmax=70)
pal = sns.diverging_palette(240, 10, n=10)
plt.figure(figsize=(20, 15))
sns.set(font_scale=0.5)
g = sns.heatmap(
    crs_tbl,
    cmap="viridis_r",
    annot=False,
    vmin=0,
    vmax=np.max(np.max(crs_tbl)),
    linewidths=1,
    center=np.max(np.max(crs_tbl)) / 2,
    square=True,
    cbar_kws={"shrink": 0.5},
)

plt.ylabel("Original labels")
plt.xlabel("SVM Classification")
# plt.savefig('./ldvae_ver5_lr_model_means_subclusters.pdf',dpi=300)
plt.show()

In [ ]:
import pickle

# Save the SVM model to a file
model_file = "svm_organoid_sk_time_nystroem_X.pkl"
with open(model_file, "wb") as file:
    pickle.dump(svm, file)

# nystroeam_file = 'nystroeam_file_X.pkl'
# with open(nystroeam_file, 'wb') as file:
#     pickle.dump(nystroem, file)
#     # Load the SVM model from the file
# with open(model_file, 'rb') as file:
#     loaded_svm = pickle.load(file)

# Convert Nystroem features back into gene expression features using the feature lookup matrix

In [ ]:
import pandas as pd
from sklearn.inspection import permutation_importance

# Compute permutation importance
perm_importance = permutation_importance(
    svm,
    nystroem_features,
    sc_labels,
    scoring="f1_weighted",
    n_repeats=10,
    random_state=42,
)

# Get feature importances for each class
feature_importances = perm_importance.importances

# Create a DataFrame to store the results
df_feature_importance = pd.DataFrame(columns=["Class", "Feature", "Importance"])

# Iterate over each class
for class_label, feature_scores in enumerate(feature_importances):
    # Get the indices of the most important features for the current class
    top_features_indices = feature_scores.argsort()[::-1]

    # Iterate over the top features
    for feature_idx in top_features_indices:
        importance = feature_scores[feature_idx]
        feature_name = f"Feature {feature_idx}"

        # Add the feature information to the DataFrame
        df_feature_importance = df_feature_importance.append(
            {"Class": class_label, "Feature": feature_name, "Importance": importance},
            ignore_index=True,
        )

# Print the DataFrame with the most important features for each class
print(df_feature_importance)

# Deconvulution loop

In [131]:
# Step 3: Load and normalize bulk RNA sequencing data
bulk_data = sc.read(
    "/nfs/team205/ig7/projects/SCC_nano_string/010523_project_restart/nanostring_geom_normalised_counts_object_040523.h5ad"
)
bulk_data = bulk_data[:, list(sc_data.var.index)]
bulk_expr = bulk_data.X
bulk_data.X = np.nan_to_num(bulk_data.X, nan=0, posinf=0)
sc.pp.normalize_per_cell(bulk_data, counts_per_cell_after=1e4)
sc.pp.log1p(bulk_data)
# sc.pp.highly_variable_genes(bulk_data, min_mean=0.1, max_mean=4)
# adata_mac = nameadata_mac adata_mac.var['highly_variable']]
sc.pp.scale(bulk_data, max_value=10)
bulk_data.X = np.nan_to_num(bulk_data.X, nan=0, posinf=0)

/home/jovyan/my-conda-envs/knn_bayes/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell
/home/jovyan/my-conda-envs/knn_bayes/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:352: RuntimeWarning: invalid value encountered in log1p
  np.log1p(X, out=X)


# Format nanostring data for bayesian hypotheses

In [132]:
bulk_data.obs["risk_stat"] = bulk_data.obs["HR_strat"].copy()
bulk_data.obs["tum.norm"] = bulk_data.obs["shortname"].copy()
bulk_stat = [
    "Bow AK",
    "HR*",
    "HR",
    "KA",
    "LOCAL RECURRANCE",
    "LR",
    "MET",
    "NEG LYMPH",
    "NON UV",
    "PERI",
]
map_stat = [
    "remove",
    "high",
    "high",
    "remove",
    "LOCAL RECURRANCE",
    "low",
    "high",
    "remove",
    "low",
    "low",
]
bulk_site = ["Disease", "Control", "peri"]
map_site = ["Tumor", "Normal", "Normal"]
status_dic = dict(zip(bulk_stat, map_stat))
site_dic = dict(zip(bulk_site, map_site))
bulk_data.obs["risk_stat"] = bulk_data.obs["risk_stat"].map(status_dic)
bulk_data.obs["tum.norm"] = bulk_data.obs["tum.norm"].map(site_dic)

bulk_data.obs[batch_id] = "patient"
bulk_data = bulk_data[~bulk_data.obs["risk_stat"].isin(["remove"])]

In [133]:
bulk_probs = pd.DataFrame()
for index in bulk_data.obs.index:
    bulk_expr_tmp = bulk_data[bulk_data.obs.index.isin([index])].X
    bulk_probs_tmp = pd.DataFrame(svm.predict_proba(nystroem.transform(bulk_expr_tmp)))
    bulk_probs_tmp.index = [index]
    bulk_probs_tmp.columns = svm.classes_
    bulk_probs = pd.concat([bulk_probs, bulk_probs_tmp])

In [134]:
bulk_data

View of AnnData object with n_obs × n_vars = 57 × 1365
    obs: 'Pathway_Name', 'Immune_Name', 'shortname', 'stage', 'tissue', 'additional', 'samplecode', 'patient', 'HR_strat', 'immunotype', 'immunocompromised', 'Sample_type', 'n_counts', 'risk_stat', 'tum.norm'
    var: 'Genes', 'Immune_panel', 'Pathway_panel', 'non-specific_genes', 'In_processed_object', 'Immune_HR', 'Immune_HR2', 'Immune_MET', 'Immune_HR3', 'Immune_HR4', 'Immune_HR5', 'Immune_MET1', 'Immune_NEG LYMPH', 'Immune_HR6', 'Immune_MET2', 'Immune_LR', 'Immune_NEG LYMPH1', 'Immune_MET3', 'Immune_NEG LYMPH2', 'Immune_Local recurrance 2', 'Immune_MET4', 'Immune_NEG LYMPH3', 'Immune_HR7', 'Immune_MET5', 'Immune_NEG LYMPH4', 'Immune_HR8', 'Immune_HR9', 'Immune_MET6', 'Immune_MET7', 'Immune_HR10', 'Immune_MET8', 'Immune_HR11', 'Immune_MET9', 'Immune_NEG LYMPH5', 'Immune_HR12', 'Immune_LR1', 'Immune_LR2', 'Immune_LR3', 'Immune_LR4', 'Immune_LR5', 'Immune_HR13', 'Immune_PERI', 'Immune_HR14', 'Immune_PERI1', 'Immune_KA', 'Immune_LR

In [137]:
(bulk_probs["TSK"] - (np.min(bulk_probs["TSK"]))) / (
    np.max(bulk_probs["TSK"]) - np.min(bulk_probs["TSK"])
)

HR                    0.160189
HR10                  0.030329
HR11                  1.000000
HR12                  0.052385
HR13                  0.052748
HR14                  0.082181
HR15                  0.030101
LR12                  0.113069
HR17                  0.063624
HR18                  0.031314
HR19                  0.040267
HR2                   0.019209
HR20                  0.035332
HR21                  0.121674
HR22                  0.450647
HR23                  0.023007
Local recurrance 1    0.064468
HR3                   0.077955
HR4                   0.042243
HR5                   0.081418
HR6                   0.203901
HR7                   0.384844
HR8                   0.232766
HR9                   0.020267
Local recurrance 2    0.055963
LR                    0.067588
LR1                   0.036681
LR10                  0.058592
LR11                  0.047342
LR2                   0.131132
LR3                   0.102267
LR4                   0.023304
LR5     

In [28]:
def deconvolve_bulk(model,bulk_data,meta_vars)
    # This function takes an input of stacked bulk data in anndata format and returns a dataframe output containing predicted cell-state contribution probability
    
    # model = SVM model
    # Bulk data = bulk data formated as anndata, assumes that each anndata.obs row contains a bulk sample
    # meta_var = sample metadata that matches between datasets to help educate bayesian priors. e.g site and disease info
    bulk_proba_mat = pd.DataFrame()
    svm = model
    for bulk_
    
    bulk_expr = bulk_data
    

SyntaxError: expected ':' (4142736286.py, line 1)

In [ ]:
bulk_data.obs

In [ ]:
bulk_probs

# Empirical bayesian framework probability refinement
- split the training data by the meta_vars (disease/site info) and extract seperate distribution information for each celltstate
- Empirical observations as we draw our priors from the sc data using a hypergeometric sampler which satisfies a chi-square test. 

#### Hypergeometric sampler
- Since we cannot assume to have captured the same level of detail in the target modaility
- We choose to sample the sc training data severaql times at x proportion to provide representative distributions corresponding to possible sets of cells which may be recovered in smaller runs


#### Sampling weights
- cell_type_distributions_per_donor is a numpy array or pandas DataFrame where each row represents a donor and each column represents a cell type. The values in the array indicate the presence or absence of each cell type in each donor.

- The num_donors_per_cell_type variable counts the number of donors in which each cell type is observed by summing the occurrences greater than 0 along the donor axis.

- The sampling_weights variable is then computed as the reciprocal of the number of donors per cell type. This assigns higher weights to cell types observed in fewer donors and lower weights to cell types observed in more donors.

- Finally, the sampling_weights are normalized to ensure they sum up to 1, allowing them to be directly used as weights in the Bayesian inference.

- You can then use the compute_sampling_weights function to compute the sampling weights and pass them to the compute_posterior_probabilities function as the sampling_weights argument.

#### Priors
- the prior distribution prior is defined using the aggregated cell type distributions obtained from multiple samples. The cell_type_distributions input should be a 2D array where each row represents the cell type distribution of a sample.

- The observed probabilities are iterated, and for each observed probability, a likelihood is defined using the Multinomial distribution with the prior distribution as the parameter. Bayesian inference is performed using the Hamiltonian Monte Carlo (HMC) algorithm via the pm.sample function. The posterior probability is then computed as the mean of the trace for the prior distribution.

- Note that this code assumes you have preprocessed the observed probabilities and cell type distributions appropriately before passing them to the compute_posterior_probabilities function.

In [ ]:
sc_data.obs

In [45]:
meta_vars = ["risk_stat", "tum.norm"]  # this could combine 'tum.norm' and sampling site
batch_id = "patient"  # This could also be 'patient'
train_var = train_var
# define the maximum number of hypergeometric samples to use in the computation of empirical priors
max_iter = 100
# Define k_samples which represents preffered number of hypergeometric samples per batch
k_samples = 10
# Define the poportion to sample with each hypergeometric pass
sampling_prop = 0.2

# Define the var_combination which matches sets of meta_vars from the SC data to assign priors
bulk_vars = ["risk_stat", "tum.norm"]

# Define all unique cell-classes
cell_states = sc_labels.unique()

############ function block ############
# Sampling empirical priors to be used in the bayesian framework
# Piors are sampled per batch and tested against the combined representation via chi-square
# Sampling is meant to represent the native sampling rate of a given phenotype representation


def hypergeometric_sampling(obs, proportion):
    """
    Perform hypergeometric sampling to select a defined proportion of cells from an AnnData object.

    Parameters:
        anndata_obj (AnnData): Annotated data object containing cell information.
        proportion (float): Proportion of cells to sample, between 0 and 1.

    Returns:
        ndarray: Selected indices.
    """
    total_cells = obs.shape[0]
    num_indices = int(proportion * total_cells)
    indices = np.arange(total_cells)

    # Calculate the probability of each index based on the total number of cells
    probabilities = np.ones(total_cells) / total_cells

    # Perform hypergeometric sampling to select indices
    selected_indices = np.random.choice(
        indices, size=num_indices, replace=False, p=probabilities
    )

    return selected_indices


def draw_emprirical_bayes_priors(
    bayesian_obs, cell_states, batch_id, train_var, max_iter, k_samples, sampling_prop
):
    # Initialise a dictionary to store the celltype distributions from each sampling set
    sampled_distributions = {}
    # initialise a counter for successful sampling and current iteration
    successes = 0
    iteration = 0
    # Disable warnings
    import warnings

    warnings.filterwarnings("ignore")

    # compute an original distribution across all cells
    p_tmp = bayesian_obs.groupby([batch_id, train_var]).apply(len).reset_index()
    # p_tmp =(p_tmp.pivot(batch_id,train_var).fillna(0)).reset_index().melt(id_vars=batch_id)
    missing_classes = list(set(list(p_tmp[train_var].unique())) ^ set((cell_states)))
    # add any missing cell-states back in
    p_tmp_pv = (p_tmp.pivot(batch_id, train_var).fillna(0)).droplevel(0, axis=1)
    p_tmp_pv[missing_classes] = 0
    p_tmp = p_tmp_pv.reset_index().melt(id_vars=batch_id)
    p_tmp["prop"] = p_tmp["value"] / p_tmp.groupby(batch_id)["value"].transform("sum")
    while successes < k_samples and iteration < max_iter:
        print("Current iteration: {}".format(iteration))

        # iteratively draw a proportion of cells from a hypergeometric distribution
        tmp = pd.DataFrame()
        for batch in bayesian_obs[batch_id].unique():
            tmp = pd.concat(
                [
                    tmp,
                    pd.DataFrame(
                        bayesian_obs[bayesian_obs[batch_id].isin([batch])]
                        .iloc[
                            hypergeometric_sampling(
                                (bayesian_obs[bayesian_obs[batch_id].isin([batch])]),
                                sampling_prop,
                            )
                        ]
                        .groupby([batch_id, train_var])
                        .apply(len)
                        .reset_index()
                    ),
                ]
            )
        missing_classes = list(
            set(list(tmp[train_var].unique())) ^ set(list(p_tmp[train_var].unique()))
        )
        # add any missing cell-states back in
        tmp_pv = (tmp.pivot(batch_id, train_var).fillna(0)).droplevel(0, axis=1)
        tmp_pv[missing_classes] = 0
        tmp = tmp_pv.reset_index().melt(id_vars=batch_id)
        tmp["prop"] = tmp["value"] / tmp.groupby(batch_id)["value"].transform("sum")
        # This is one sampling loop, ensure that samples from this loop are representative of the main data
        # Take the mean probability of succeesfully sampled data between batches,per loop and test with chi-square test

        # Create a contingency table with the original and sampled distributions
        observed = np.array([tmp["prop"]])
        expected = np.array([p_tmp["prop"]])
        # Create contingency table for original and sampled distributions
        contingency_table = pd.crosstab(expected, observed)
        # Add a small constant value to expected frequencies
        contingency_table += 0.001

        # Perform chi-square test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        print("Chi-square statistic: {} , p-value: {}".format(chi2, p_value))
        # If the sampling loop is successful (p<0.05), we add it to the observed priors list
        # We take the successfully sampled probabilities as input for priors
        if p_value < 0.05:
            successes = successes + 1
            sampled_distributions[iteration] = tmp
            # Print the chi-square statistic and p-value

        iteration = iteration + 1
        if iteration == max_iter:
            print("max iteration hit, proceeding anyways")
        else:
            print(
                "Suceeded in drawing k= {} hypergeometric samples using defined proportion = {} in n_iter = {} iterations".format(
                    k_samples, sampling_prop, iteration
                )
            )
    warnings.filterwarnings("default")
    return sampled_distributions


def estimate_posterior_probabilities(priors, observations):
    num_samples, num_cell_types = observations.shape

    with pm.Model() as model:
        # Priors for cell type contributions
        cell_type_contributions = pm.Dirichlet(
            "cell_type_contributions", a=priors, shape=num_cell_types
        )

        # Likelihood of observations
        for i in range(num_samples):
            pm.Bernoulli(
                "likelihood_" + str(i),
                p=cell_type_contributions,
                observed=observations[i],
            )

        # Sample from the posterior distribution
        trace = pm.sample(draws=2000, tune=1000)
        # Extract posterior probabilities
        posteriors = pd.DataFrame(
            pm.trace_to_dataframe(trace, varnames=["cell_type_contributions"])
        )
    return posteriors


# sampling_weights is computed as the reciprocal of the number of donors per cell type
# Assigns higher weights to cell types observed in more donors and lower weights to cell types observed in fewer donors
# The idea here is to find common celltypes across donors which are representative of the state=
def compute_sampling_weights(cell_type_distributions_per_donor):
    # In this code, cell_type_distributions_per_donor is a numpy array or pandas DataFrame where each row represents a donor and each column represents a cell type. The values in the array indicate the presence or absence of each cell type in each donor.
    # The num_donors_per_cell_type variable counts the number of donors in which each cell type is observed by summing the occurrences greater than 0 along the donor axis.
    # The sampling_weights variable is then computed as the reciprocal of the number of donors per cell type. This assigns higher weights to cell types observed in more donors and lower weights to cell types observed in fewer donors.
    # Finally, the sampling_weights are normalized to ensure they sum up to 1, allowing them to be directly used as weights in the Bayesian inference.
    # You can then use the compute_sampling_weights function to compute the sampling weights and pass them to the compute_posterior_probabilities function as the sampling_weights argument.

    # The idea here is to find common celltypes across donors which are representative of the state

    # Count the number of donors in which each cell type is observed
    num_donors_per_cell_type = np.sum(cell_type_distributions_per_donor > 0, axis=0)
    # Compute the sampling weights based on the number of donors
    sampling_weights = 1 / num_donors_per_cell_type
    sampling_weights[sampling_weights == inf] = 0
    # Normalize the sampling weights to sum up to 1
    sampling_weights /= np.sum(sampling_weights)
    return sampling_weights


############ function block ############

# create a loop which grabs unique combinations of each meta var (risk strat and site)
var_combi = adata.obs[meta_vars].value_counts().reset_index(name="count")
var_combi["combi"] = var_combi[meta_vars].apply(
    lambda row: "_".join(row.values.astype(str)), axis=1
)
obs = adata.obs.copy()
obs["bayesian_prior_meta"] = obs[meta_vars].apply(
    lambda row: "_".join(row.values.astype(str)), axis=1
)

for var_combi_idx in var_combi.index:
    combi = var_combi.loc[var_combi_idx, "combi"]
    # Grab data which matches a metadata class of patient stratification, maybe include sampling region too
    bayesian_obs = obs[obs["bayesian_prior_meta"].isin([combi])]
    # if lenght is <3, consider passing without liklihood correction

# Draw empirical samples for pirors
bayesian_obs = obs[obs["bayesian_prior_meta"].isin(["high_Tumor"])]
empirical_priors = draw_emprirical_bayes_priors(
    bayesian_obs, cell_states, batch_id, train_var, max_iter, k_samples, sampling_prop
)
priors_df = pd.DataFrame()
for key in empirical_priors.keys():
    prior = (
        empirical_priors[key].pivot(batch_id, train_var, values="prop").fillna(0)
    )  # .droplevel(0,axis =1)
    prior.index = prior.index.astype(str) + "_" + str(key)
    priors_df = pd.concat([priors_df, prior])

# Compute sampling weights
# - Weights are celltype availability per donor from the original distribution
# - We penalise celltype probabilities which are represented in fewer donors
p_tmp = bayesian_obs.groupby([batch_id, train_var]).apply(len).reset_index()
p_tmp = (p_tmp.pivot(batch_id, train_var).fillna(0)).droplevel(0, axis=1)
# compute an original distribution across all cells
p_tmp = bayesian_obs.groupby([batch_id, train_var]).apply(len).reset_index()
# p_tmp =(p_tmp.pivot(batch_id,train_var).fillna(0)).reset_index().melt(id_vars=batch_id)
missing_classes = list(set(list(p_tmp[train_var].unique())) ^ set((cell_states)))
# # add any missing cell-states back in
# p_tmp_pv =(p_tmp.pivot(batch_id,train_var).fillna(0)).droplevel(0,axis =1)
# p_tmp_pv[missing_classes] = 0
# p_tmp_pv

# Create sampling weights
p_tmp = bayesian_obs.groupby([batch_id, train_var]).apply(len).reset_index()
p_tmp = (p_tmp.pivot(batch_id, train_var).fillna(0)).droplevel(0, axis=1)
# compute an original distribution across all cells
p_tmp = bayesian_obs.groupby([batch_id, train_var]).apply(len).reset_index()
# p_tmp =(p_tmp.pivot(batch_id,train_var).fillna(0)).reset_index().melt(id_vars=batch_id)
missing_classes = list(set(list(p_tmp[train_var].unique())) ^ set((cell_states)))
# add any missing cell-states back in
p_tmp_pv = (p_tmp.pivot(batch_id, train_var).fillna(0)).droplevel(0, axis=1)
p_tmp_pv[missing_classes] = 0
p_tmp = p_tmp_pv

sampling_weights = compute_sampling_weights(p_tmp)

# Bayesian HMC
priors = pd.DataFrame(bulk_probs)
observations = priors_df
posteriors = estimate_posterior_probabilities(
    np.array(priors).flatten(), observations.values
)
posteriors.columns = observations.columns
# posteriors = pd.DataFrame(posteriors,columns=priors.columns)
print(posteriors)
print(posteriors.mean(0))

Current iteration: 0
Chi-square statistic: 9743.76344235169 , p-value: 3.780426359559009e-18
Suceeded in drawing k= 10 hypergeometric samples using defined proportion = 0.2 in n_iter = 1 iterations
Current iteration: 1
Chi-square statistic: 9895.22786877249 , p-value: 8.327922857105144e-19
Suceeded in drawing k= 10 hypergeometric samples using defined proportion = 0.2 in n_iter = 2 iterations
Current iteration: 2
Chi-square statistic: 9620.045605559972 , p-value: 5.187923477346367e-15
Suceeded in drawing k= 10 hypergeometric samples using defined proportion = 0.2 in n_iter = 3 iterations
Current iteration: 3
Chi-square statistic: 9057.946806818076 , p-value: 1.282628518578592e-13
Suceeded in drawing k= 10 hypergeometric samples using defined proportion = 0.2 in n_iter = 4 iterations
Current iteration: 4
Chi-square statistic: 9244.523252151628 , p-value: 2.2150863218889413e-18
Suceeded in drawing k= 10 hypergeometric samples using defined proportion = 0.2 in n_iter = 5 iterations
Curren

/tmp/ipykernel_2210/918053593.py:163: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  prior = (empirical_priors[key].pivot(batch_id,train_var,values = 'prop').fillna(0))#.droplevel(0,axis =1)
/tmp/ipykernel_2210/918053593.py:163: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  prior = (empirical_priors[key].pivot(batch_id,train_var,values = 'prop').fillna(0))#.droplevel(0,axis =1)
/tmp/ipykernel_2210/918053593.py:163: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  prior = (empirical_priors[key].pivot(batch_id,train_var,values = 'prop').fillna(0))#.droplevel(0,axis =1)
/tmp/ipykernel_2210/918053593.py:163: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  prior = (empirical_priors[key].pivot(batch_id,train_var,values = 'prop').fillna(0))#.droplevel(0,axis =1)
/tmp/ipykernel_2

# bayesian imputation of gene expression profile per cell-state
A Bayesian framework for imputing gene expression in bulk data using single-cell data involves modeling the gene expression values as a Gaussian process. The basic idea is to model the latent gene expression values in the bulk data as a function of the latent gene expression values in the single-cell data, and use a Bayesian approach to estimate the parameters of the Gaussian process. The resulting model can be used to predict the gene expression values for each cell type in the bulk data.

Let's define some notation. Let $y_b$ be the observed gene expression values for a bulk sample, $X_b$ be the design matrix for the bulk sample, and $\beta_b$ be the coefficients in a linear regression model relating the gene expression values in the bulk sample to the latent gene expression values in the single-cell data. Let $y_s$ be the observed gene expression values for a single-cell sample, $X_s$ be the design matrix for the single-cell sample, and $\beta_s$ be the coefficients in a linear regression model relating the gene expression values in the single-cell sample to the latent gene expression values.

We can model the latent gene expression values in the bulk sample as a Gaussian process with a mean function that is a linear combination of the latent gene expression values in the single-cell sample:

$$ f_b \sim \mathcal{GP}(\mathbf{X}_b\beta_b, k_b(\mathbf{Z},\mathbf{Z})), $$

where $\mathbf{Z}$ is the matrix of latent gene expression values in the single-cell sample, $k_b(\cdot,\cdot)$ is a kernel function for the bulk sample, and $\beta_b$ is a vector of coefficients that relate the latent gene expression values in the single-cell sample to the observed gene expression values in the bulk sample.

We can then use a Bayesian approach to estimate the parameters of the Gaussian process. Specifically, we can compute the posterior distribution of the latent gene expression values in the bulk sample given the observed gene expression values and the latent gene expression values in the single-cell sample:

$$ p(\mathbf{f}_b | \mathbf{y}_b, \mathbf{Z}) \propto p(\mathbf{y}_b | \mathbf{f}_b) p(\mathbf{f}_b | \mathbf{X}_b, \mathbf{Z}), $$

where $p(\mathbf{y}_b | \mathbf{f}_b)$ is the likelihood of the observed gene expression values given the latent gene expression values, and $p(\mathbf{f}_b | \mathbf{X}_b, \mathbf{Z})$ is the prior distribution of the latent gene expression values.

The posterior distribution can be computed using the standard Bayesian inference formula:

$$ p(\mathbf{f}_b | \mathbf{y}_b, \mathbf{Z}) = \frac{p(\mathbf{y}_b | \mathbf{f}_b) p(\mathbf{f}_b | \mathbf{X}_b, \mathbf{Z})}{p(\mathbf{y}_b | \mathbf{X}_b, \mathbf{Z})}, $$

where $p(\mathbf{y}_b | \mathbf{X}_b, \mathbf{Z})$ is the marginal likelihood of the observed gene expression values.

Once we have computed the posterior distribution of the latent gene expression values in the bulk sample, we can use it to predict the gene expression values for each cell type in the bulk sample:

$$ \hat{\mathbf{y}}_b^{(c)} = \mathbf{X}_b

### 
In this section of the code, we define a function called impute_gene_expression() that performs gene expression imputation for a specific cell type using a Bayesian framework. The function takes as input a boolean array called ct_mask that indicates which cells in the single-cell reference data are of the target cell type, a Pandas DataFrame called bulk_expr that contains the bulk RNA sequencing data, a trained SVM classifier called clf, and prior mean and variance arrays called prior_mean and prior_var. The function first fits the SVM classifier to the single-cell reference data for the target cell type using the fit() method. It then computes the posterior mean and variance for each gene in the target cell type using the Bayesian framework described earlier. Finally, it imputes the gene expression values for the target cell type using the posterior mean and variance, and returns them as a Pandas DataFrame.

After defining the impute_gene_expression() function, we initialize the prior mean and variance arrays using the mean and variance of the gene expression values in the single-cell reference data. We then initialize an empty dictionary called imputed_expr_dict to store the imputed gene expression values for each cell type.

```

In this section of the code, we define a function called `impute_gene_expression()` that performs gene expression imputation for a specific cell type using a Bayesian framework. The function takes as input a boolean array called `ct_mask` that indicates which cells in the single-cell reference data are of the target cell type, a Pandas DataFrame called `bulk_expr` that contains the bulk RNA sequencing data, a trained SVM classifier called `clf`, and prior mean and variance arrays called `prior_mean` and `prior_var`. The function first fits the SVM classifier to the single-cell reference data for the target cell type using the `fit()` method. It then computes the posterior mean and variance for each gene in the target cell type using the Bayesian framework described earlier. Finally, it imputes the gene expression values for the target cell type using the posterior mean and variance, and returns them as a Pandas DataFrame.

After defining the `impute_gene_expression()` function, we initialize the prior mean and variance arrays using the mean and variance of the gene expression values in the single-cell reference data. We then initialize an empty dictionary called `imputed_expr_dict` to store the imputed gene expression values for each cell type.

We then iterate over each

In [ ]:
# Gene expression imputation using Bayesian framework


# First, we define a function to compute the gene expression imputation for a given cell type
def impute_gene_expression(ct_mask, bulk_expr, clf, prior_mean, prior_var):
    """
    Imputes gene expression values for a specific cell type using a Bayesian framework.

    Args:
        ct_mask (np.array): Boolean array indicating which cells in the single-cell reference data are of the target cell type
        bulk_expr (pd.DataFrame): Bulk RNA sequencing data
        clf (sklearn.svm.SVC): Trained support vector machine classifier
        prior_mean (np.array): Prior mean for Bayesian imputation
        prior_var (np.array): Prior variance for Bayesian imputation

    Returns:
        pd.DataFrame: Imputed gene expression values for the target cell type in the bulk RNA sequencing data
    """

    # Fit SVM classifier to single-cell reference data for the target cell type
    clf.fit(sc_expr[ct_mask], np.ones(np.sum(ct_mask)))

    # Compute posterior mean and variance for each gene in the target cell type
    # using Bayesian framework
    posterior_mean = prior_mean + clf.decision_function(bulk_expr) @ clf.dual_coef_.T
    posterior_var = prior_var - np.sum(clf.coef_**2, axis=0) / (2 * clf.C)

    # Impute gene expression values using the posterior mean and variance
    imputed_expr = np.random.normal(loc=posterior_mean, scale=np.sqrt(posterior_var))

    # Return imputed gene expression values as a dataframe
    return pd.DataFrame(imputed_expr, columns=bulk_expr.columns)


# Define prior mean and variance for Bayesian imputation
prior_mean = sc_data.X.mean(axis=0)
prior_var = sc_data.X.var(axis=0)

# Initialize dictionary to store imputed gene expression values for each cell type
imputed_expr_dict = {}

# Iterate over each cell type
for ct in cell_types:
    ct_mask = sc_data.obs["level3_celltype"] == ct

    # Compute imputed gene expression values for the cell type using the previously defined function
    imputed_expr = impute_gene_expression(
        ct_mask, bulk_expr, clf, prior_mean, prior_var
    )

    # Store imputed gene expression values in dictionary
    imputed_expr_dict[ct] = imputed_expr

SyntaxError: invalid syntax (1908105408.py, line 49)

# Probability description

In this case, the probabilistic SVM classifier is trained using all single-cell gene expression data, treating each cell as a separate binary classification problem, with positive labels indicating that the cell belongs to a particular cell type, and negative labels indicating that it does not. The SVM model then learns a decision boundary that separates the positive and negative samples in the feature space, with the goal of maximizing the margin between the two classes while minimizing the classification error.

Once the SVM model is trained, it can be used to predict the probability that a new sample (e.g., bulk RNA-seq data) belongs to each cell type. This is done by computing the distance of the new sample to the decision boundary, and then transforming this distance into a probability estimate using a sigmoid function, as described in my earlier response.

Therefore, instead of training separate SVM classifiers for each cell type, a single classifier is trained on all the data, with each cell type treated as a separate class.


The distance of a new sample to the decision boundary in an SVM classifier is typically computed using the signed distance function. This function measures the distance of the sample to the decision boundary, and is positive if the sample is on the positive side of the boundary (i.e., closer to the positive class) and negative if the sample is on the negative side of the boundary (i.e., closer to the negative class). The signed distance function can be expressed as:

d(x)= i=1∑N sv​​α i​y i​ K(x,x i​)+b

where $d(\mathbf{x})$ is the signed distance of the sample $\mathbf{x}$ to the decision boundary, $N_{sv}$ is the number of support vectors, $\alpha_i$ and $y_i$ are the Lagrange multipliers and labels of the support vectors, respectively, $K(\mathbf{x}, \mathbf{x_i})$ is the kernel function evaluated at the sample and support vector $\mathbf{x_i}$, and $b$ is the bias term.

The signed distance function can then be transformed into a probability estimate using a sigmoid function.

#### Why the isotonic regressor method is not used:

The isotonic regression method is a different approach that can be used for calibration of probabilistic predictions. It is used to transform a set of uncalibrated probability estimates into calibrated probabilities that are more reliable and better calibrated with respect to the true probability. In contrast, the signed distance function in an SVM classifier is not a probabilistic prediction, but rather a measure of the distance of a sample to the decision boundary.


The isotonic regression method is not directly applicable for the task of probabilistic deconvolution of bulk RNA-seq data using single-cell RNA-seq data as input. The main purpose of isotonic regression is to transform uncalibrated probability estimates into calibrated probabilities that are more reliable and better calibrated with respect to the true probability.

In the workflow described, the SVM classifier already outputs calibrated probability estimates for the different cell types, which are obtained by transforming the signed distance function using a sigmoid function. Therefore, there is no need to apply isotonic regression for calibration of the probability estimates.

However, isotonic regression may be useful in other contexts where calibration of probability estimates is required, such as in classification tasks where the output of the classifier is a set of uncalibrated probability estimates. In such cases, isotonic regression can be used to transform the probability estimates into calibrated probabilities that are more reliable and better calibrated with respect to the true probability.

d(x)= i=1∑N sv​​α i​y i​ K(x,x i​)+b
